# 311 complaints dataset

#### Author: Francesco Ciraolo

## Dataset origin
https://data.boston.gov/dataset/311-service-requests


In [1]:
import dataset_311_downloader

from folium import Icon, LayerControl, Map, Marker
from folium.plugins import HeatMap, MarkerCluster

import pandas as pd
import os
import re

In [2]:
__BE_STRICT = True

In [3]:
df311 = dataset_311_downloader.download_and_load_datasets()

/home/fciraolo/Projects/ds-councilor-breadon-bad-landlords2/spring23-team-4/code/dataset_311_downloader.py:66: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(source) for source in sources]


In [4]:
df311 = df311[df311['case_status'] == 'Closed']
df311 = df311[df311['reason'] == 'Housing']
df311['closure_reason'] = df311.closure_reason\
    .map(lambda c: re.sub('\s*[Cc]losed [Dd]ate : (?:(?:\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}.\d{,3})|(?:\w{3} \w{3} \d\d \d\d:\d\d:\d\d \w{3} \d{4}))(?: :)?\s*', '', re.sub('\s*[Cc]ase [Cc]losed\s*\.?:?\s*', '', c)))

In [5]:
_short_codes = set()

for x in df311.closure_reason.unique():
    m = re.match('([A-Z]*):.*', x)
    if m:
        _short = m.group(1)
        if _short not in _short_codes:
            _short_codes.add(m.group(1))
            # print(x)

keywords = [('resolved', 'Resolved'), ('duplicate', 'Duplicated'), ('invalid', 'Invalid'), ('noted', 'Noted'), ('resubmitted', 'Resubmitted'), ('external agency', 'External'), ('withdrawn', 'Withdrawn')]

def closure_status(reason: str):

    if not reason:
        return 'Undefined'

    for code in _short_codes:
        if code in reason:
            return code

    reason = reason.lower()

    for t, v in keywords:
        if t in reason:
            return v

    return None

df311['closure_status'] = df311.closure_reason.map(closure_status)

In [6]:
rentsmart_df = pd.read_csv('../data/rentsmart_map_short.csv', usecols=['violation_type', 'latitude', 'longitude'])
rentsmart_df.columns = ['label', 'lat', 'lon']
rentsmart_df

,label,lat,lon
0,Housing Complaints,42.28120,-71.07215
1,Housing Complaints,42.34554,-71.08974
2,Housing Complaints,42.34554,-71.08974
3,Housing Complaints,42.29957,-71.06851
4,Housing Complaints,42.32505,-71.09353
...,...,...,...
54924,Housing Complaints,42.31128,-71.07707
54925,Housing Complaints,42.31128,-71.07707
54926,Housing Complaints,42.30350,-71.06456
54927,Housing Complaints,42.35921,-71.06688


In [7]:
property_violations_df = pd.read_csv('../data/public_health_violations_lat_long.csv')
property_violations_df.columns = ['label', 'lat', 'lon']
property_violations_df

,label,lat,lon
0,CMR410.482 A,42.311610,-71.092340
1,CMR410.500,42.311610,-71.092340
2,CMR410.482 A,42.311610,-71.092340
3,CMR410.482 A,42.311610,-71.092340
4,CMR410.500,42.311610,-71.092340
...,...,...,...
24335,110.1,42.375200,-71.032310
24336,110.1,42.337380,-71.075270
24337,5110.1,42.278718,-71.134019
24338,300.15,42.349028,-71.061613


In [15]:
def plot_dataset(df, title, base_map, color='green'):

    _group = MarkerCluster(name=title).add_to(base_map)

    for _, data in df.iterrows():
        Marker(location=[data['lat'], data['lon']], icon=Icon(color=color)).add_to(_group)

    # LayerControl().add_to(base_map)

In [16]:
def plot_top_n_violations_sl(df, base_map, n=5, strict=__BE_STRICT):
    _tmp = df[~df.closure_status.isin(['NOVIO', 'Duplicated', 'Resubmitted', 'Withdrawn', 'NOBASE', 'CLOSED', 'NOACC', None] + ['Undefined'] if strict else [])]
    _top = df.type.value_counts()[:n].index.tolist()

    _df = df[df.type.isin(_top)][['latitude', 'longitude', 'type']]

    _layers_dict = dict([(_top[i], i) for i in range(n)])
    # _groups = [MarkerCluster(name=_top[i]).add_to(base_map) for i in range(n)]
    _group = MarkerCluster(name=f'''Top {n} 311 dataset''').add_to(base_map)

    colors = ['red', 'orange', 'green', 'blue', 'pink']

    for _, data in _df.iterrows():
        lat, lon, type = data
        i = _layers_dict[type]
        Marker(location=[lat, lon], icon=Icon(color='orange')).add_to(_group)

    # LayerControl().add_to(base_map)

In [10]:
def plot_top_n_violations(df, base_map, n=5, strict=__BE_STRICT):
    _tmp = df[~df.closure_status.isin(['NOVIO', 'Duplicated', 'Resubmitted', 'Withdrawn', 'NOBASE', 'CLOSED', 'NOACC', None] + ['Undefined'] if strict else [])]
    _top = df.type.value_counts()[:n].index.tolist()

    _df = df[df.type.isin(_top)][['latitude', 'longitude', 'type']]

    _layers_dict = dict([(_top[i], i) for i in range(n)])
    _groups = [MarkerCluster(name=_top[i]).add_to(base_map) for i in range(n)]

    colors = ['red', 'orange', 'green', 'blue', 'pink']

    for _, data in _df.iterrows():
        lat, lon, type = data
        i = _layers_dict[type]
        Marker(location=[lat, lon], icon=Icon(color=colors[i])).add_to(_groups[i])

    LayerControl().add_to(base_map)

In [7]:
from geopy import Nominatim
class AddressToCoordinates:

    def __init__(self, user_agent='undefined'):
        self.user_agent = user_agent
        self.geolocator = Nominatim(user_agent=user_agent)

    def get_coordinates(self, address):
        location = self.geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else (None, None)

atc = AddressToCoordinates()

In [12]:
source_path = '../data/affordable_rental_units.csv'

if not os.path.exists(source_path):
    from bs4 import BeautifulSoup
    import requests

    source_url = 'https://www.bostonplans.org/housing/finding-housing/property-listings'
    page_text = requests.get(source_url).text
    del source_url

    soup = BeautifulSoup(page_text, 'html.parser')
    del page_text


    spreadsheet_url = soup.find_all('iframe')[-1].get('src')
    spreadsheet_url = '/'.join(str(spreadsheet_url).split('/')[:-1] + ['pub?output=csv'])

    spreadsheet_content = requests.get(spreadsheet_url, allow_redirects=True).text
    spreadsheet_content = re.sub('â\x80\x93', '-', spreadsheet_content)

    with open(source_path, 'w') as file:
        file.write(spreadsheet_content)

    del spreadsheet_url
    del file
    del spreadsheet_content

source_pd = pd.read_csv(source_path, skiprows=1)
# Rename columns
source_pd.columns = ['Neighborhood',
                     'Address',
                     'Info',
                     'Units',
                     'Sizes',
                     'AMI']
#Drop info
source_pd = source_pd[['Neighborhood',
                     'Address',
                     'Units',
                     'Sizes',
                     'AMI']]

# Weak assumption
source_pd['Units'] = source_pd['Units'].fillna(1)

source_pd = source_pd.dropna()

def explode_address(address):
    _old = address
    if '\n' in address:
        _complex, address = address.split('\n')
    else:
        m = re.search(r'^(?:(.*?)??[ ,]*\s)?(\d+(?:\s*[-&]\s*\d+)?\s*.+)', address)
        if m:
            _complex = m.groups()[0] if len(m.groups()) > 1 else None
            address = m.groups()[1]
      #      print(f'M Complex: {str(_complex):30} Address: {str(address):30} Old: {_old}\n{m.groups()}\n')
        else:
            _complex = address
            address = None
     #       print(f'N Complex: {_complex:30} Address: {str(address):30} Old: {_old}\n')


    #return _complex, address, None, None
    if _complex:
        lat, lon = atc.get_coordinates(_complex + ' Boston')
    else:
        lat = None

    if not lat:
        if address:
            m = re.search('(\d+)\s*[-&]\s*\d+\s*(.*)', address)
            if m:
                addr = f'{m.group(1)} {m.group(2)}'
            else:
                addr = address

        if address:
            lat, lon = atc.get_coordinates(addr + ' Boston')

    if not lat:
        lat = lon = None

    return _complex, address, lat, lon

#source_pd[['Complex', 'Address']] = source_pd.apply(lambda r: (None, r['Address']) if not '\n' in r['Address'] else r['Address'].split('\n'), axis='columns', result_type='expand')
source_pd[['Complex', 'Address', 'Latitude', 'Longitude']] = source_pd.apply(lambda r: explode_address(r['Address']), axis='columns', result_type='expand')

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=315+on+A+Apartments+Boston&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [8]:
len(df311[~df311.closure_status.isin(['NOVIO', 'Duplicated', 'Resubmitted', 'Withdrawn', 'NOBASE', 'CLOSED', 'NOACC', None] + ['Undefined'] if __BE_STRICT else [])])

17249

In [17]:
base_map = Map(location=[42.31317164, -71.069], zoom_start=13)
# HeatMap(source_pd[source_pd.Latitude.notnull()][['Latitude', 'Longitude', 'Units']], name='Affordable renting units').add_to(base_map)
plot_top_n_violations_sl(df311, base_map)
print(1)
plot_dataset(rentsmart_df, 'Rentsmart', base_map, 'pink')
print(2)
plot_dataset(property_violations_df, 'Property violations', base_map, 'blue')
print(3)
LayerControl().add_to(base_map)
# base_map

1
2
3


In [18]:
base_map.save('map.html')

In [ ]:
tmp = df311[~df311.latitude.isna()][['latitude', 'longitude']].merge(source_pd[~source_pd.Latitude.isna()][['Latitude', 'Longitude']], how='cross')

In [ ]:
import geopy
from geopy.distance import distance
len(tmp)

In [ ]:
_res = tmp.apply(lambda t: distance((t.latitude, t.longitude), (t.Latitude, t.Longitude)).m < 10, axis=1)